In [1]:
!pip install networkx

In [2]:
import networkx as nx
from operator import itemgetter

In [3]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import numpy as np
from scipy import linalg as LA
import random
import time
import csv

In [18]:
class HITS:
  '''
  Class for implementing the HITSalgorithm
  Class variables:
    query: list containing the word tokens in the query string
    G: networkx graph (web graph)
    adj: adjacency matrix of G
    A: adjacency matrix of sub graph of G
    root_set: root set of the query
    base_set: base set of the query
    base_edges: edges present in the sub graph of G (edges b/w nodes of base_set)
    hub_score: list containing the hub scores of the nodes
    auth_score: list containing the auth scores of the nodes
  '''
  def __init__(self, G):
    '''Function for taking input and initializing variables'''

    # inputting the query and converting it to lower case
    self.query = input("Please enter the query: ").lower()
    # Tokenizing the query into words and storing them as a list
    self.query = word_tokenize(self.query)
    print("Query: ", self.query, "\n")

    self.G = G
    # l = number of nodes in the web graph
    l = len(self.G.nodes)

    # initilizing the adj matrix to be of size l*l and containig 0s
    self.adj = np.zeros((l, l))

    self.A = []

    self.root_set = []
    self.base_set = []

    self.base_edges = []

    self.hub_scores = []
    self.auth_scores = []

  def initialize_adj(self):
    '''Function to change the adjacent matrix according to the edges in G'''
    for edge in self.G.edges:
      s = edge[0]
      d = edge[1]
      self.adj[s][d] = 1

  def find_root(self):
    '''Function to find the root set'''
    for i in range(0, len(self.G.nodes)):
      flag = True
      for word in self.query:
        if word not in self.G.nodes[i]['page_content'].lower():
          flag = False
          break
      if flag == True:
        self.root_set.append(i)

    print("ROOT SET: ", self.root_set, "\n")

  def find_base(self):
    '''Function to find the base set'''
    
    # adding the nodes in the root_set to the base_set
    for node in self.root_set:
      self.base_set.append(node)
    
    # adding nodes connected to the root_set nodes into the base set
    for edge in self.G.edges:
      src = edge[0]
      dest = edge[1]
      if src in self.root_set:
        if dest not in self.base_set:
          self.base_set.append(dest)
      elif dest in self.root_set:
        if src not in self.base_set:
            self.base_set.append(src)
    
    print("BASE SET: ", self.base_set, "\n")
    
    # initializing adjacency matrix of sub graph to be of size l*l (l=size(base_set)) and to contain all 0s initially
    l = len(self.base_set)
    self.A = np.zeros((l,l))
    
    for i in self.base_set:
      for j in self.base_set:
        if self.adj[i][j] == 1:
          s = self.base_set.index(i)
          d = self.base_set.index(j)
          self.A[s][d] = 1
          # storing the edges present in subgraph in base_edges
          self.base_edges.append((i,j))

  def find_hub_score(self):
    '''Function to find the hub scores of the nodes'''

    aaT = np.dot(self.A, self.A.T)
    
    # finding the eigenvector of (A.Atranspose) using the linear alg library
    v, V= LA.eig(a=np.array(aaT),b=None,left=True,right=False,overwrite_a=False,overwrite_b=False,check_finite=False)

    # v, V = LA.eig(aaT)
    # self.hub_scores = V[:, 0].T

    # hub_scores = principle left eigen vector of (A.Atranspose)
    max_eig_val = 0
    max_eig_val_ind = 0
    for i in range(len(v)):
      if (v[max_eig_val_ind] < v[i]):
        max_eig_val_ind = i
        max_eig_val = v[i]

    self.hub_scores = V[:, max_eig_val_ind]
    self.hub_scores = self.hub_scores/LA.norm(self.hub_scores)

    # normalizing the hub scores
    self.hub_scores = self.hub_scores/sum(self.hub_scores)

    result = []
    for i in range(len(self.base_set)):
      result.append((self.base_set[i], self.hub_scores[i]))
    result=sorted(result,key=itemgetter(1),reverse=True)
    print("Page Rankings:\nPage No\tHub score")
    index=1
    for x in result:
      # if index == 10:
      #   break
      for j in x:
        print(j,end="\t")
      index=index+1
      print("\n")

    # print("HUB SCORES: ")
    # for i in range(len(self.base_set)):
    #   print(self.base_set[i], ":", self.hub_scores[i])
    # print("\nSum of Hub scores: ", sum(self.hub_scores), "\n")

  def find_auth_score(self):
    '''Function to find the auth scores of the nodes'''
    aTa = np.dot(self.A.T, self.A)
    # finding the eigenvector of (Atranspose.A) using the linear alg library
    # v, V = LA.eig(aTa)
    # self.auth_scores = V[:, 0].T
    v, V= LA.eig(a=np.array(aTa),b=None,left=True,right=False,overwrite_a=False,overwrite_b=False,check_finite=False)

    # auth_scores = principle left eigen vector of (Atranspose.A)
    max_eig_val = 0
    max_eig_val_ind = 0
    for i in range(len(v)):
      if (v[max_eig_val_ind] < v[i]):
        max_eig_val_ind = i
        max_eig_val = v[i]

    self.auth_scores = V[:, max_eig_val_ind]
    self.auth_scores = self.auth_scores/LA.norm(self.auth_scores)

    # normalizing the auth scores
    self.auth_scores = self.auth_scores/sum(self.auth_scores)

    result = []
    for i in range(len(self.base_set)):
      result.append((self.base_set[i], self.auth_scores[i]))
    result=sorted(result,key=itemgetter(1),reverse=True)
    print("Page Rankings:\nPage No\tAuth score")
    index=1
    for x in result:
      # if index == 10:
        # break
      for j in x:
        print(j,end="\t")
      index=index+1
      print("\n")

    # print("AUTH SCORES: ")
    # for i in range(len(self.base_set)):
    #   print(self.base_set[i], ":", self.auth_scores[i])
    # print("\nSum of Auth scores: ", sum(self.auth_scores), "\n")

  def validating(self):
    '''Function to compare results obtained from 
    find_auth_score and find_hub_score with results obtained by using the library function
    '''
    h, a = nx.hits(self.G.subgraph(self.base_set))
    print("HUB SCORES: ")
    for i in range(len(self.base_set)):
      print(self.base_set[i], ": ", self.hub_scores[i], "  ", h[self.base_set[i]])
    # print(h)
    print("\nAUTH SCORES: ")
    for i in range(len(self.base_set)):
      print(self.base_set[i], ": ", self.auth_scores[i], "  ", a[self.base_set[i]])
    # print(a)
    
  def run(self):
    '''Function to run the algorithm'''
    self.initialize_adj()
    self.find_root()
    self.find_base()
    if (len(self.base_set) == 0):
      print("Base set is empty\n")
      return
    self.find_hub_score()
    self.find_auth_score()

    # print("\nValidating...")
    # self.validating()

# ###########     ANALYSIS     #####################################################
  def find_hub_score_analysis(self):
    aaT = np.dot(self.A, self.A.T)
    v, V= LA.eig(a=np.array(aaT),b=None,left=True,right=False,overwrite_a=False,overwrite_b=False,check_finite=False)
    max_eig_val = 0
    max_eig_val_ind = 0
    for i in range(len(v)):
      if (v[max_eig_val_ind] < v[i]):
        max_eig_val_ind = i
        max_eig_val = v[i]
    self.hub_scores = V[:, max_eig_val_ind]
    self.hub_scores = self.hub_scores/LA.norm(self.hub_scores)
    self.hub_scores = self.hub_scores/sum(self.hub_scores)

  def find_auth_score_analysis(self):
    aTa = np.dot(self.A.T, self.A)
    v, V= LA.eig(a=np.array(aTa),b=None,left=True,right=False,overwrite_a=False,overwrite_b=False,check_finite=False)
    max_eig_val = 0
    max_eig_val_ind = 0
    for i in range(len(v)):
      if (v[max_eig_val_ind] < v[i]):
        max_eig_val_ind = i
        max_eig_val = v[i]
    self.auth_scores = V[:, max_eig_val_ind]
    self.auth_scores = self.auth_scores/LA.norm(self.auth_scores)
    self.auth_scores = self.auth_scores/sum(self.auth_scores)

  def analysis(self):
    csvfile = open("HITS_Analysis.csv", "w")
    fields = ["Number of edges", "Runtime"]
    rows = []
    n = len(self.G.nodes)
    for i in range(50):
      start = time.time()
      b = random.randint(1,n)
      self.base_set = []
      for j in range(1,b):
        a = random.randint(1,n)
        self.base_set.append(a)
      self.find_hub_score_analysis()
      self.find_auth_score_analysis()
      end = time.time()
      runtime = end-start
      edges = len(self.G.subgraph(self.base_set).edges)
      rows.append([edges, runtime])
    rows.sort()
    csvwriter = csv.writer(csvfile) 
    # writing the fields 
    csvwriter.writerow(fields) 
    # writing the data rows 
    csvwriter.writerows(rows)
    csvfile.close()


In [13]:
def driver():
  '''driver code'''
  # web_graph_file = '/content/drive/MyDrive/4-2/web_graph.gpickle'
  web_graph_file = '/content/web_graph.gpickle'
  G = nx.read_gpickle(web_graph_file)
  hits = HITS(G)
  hits.run()
  # hits.analysis()

In [23]:
driver()

Please enter the query: gunfire
Query:  ['gunfire'] 

ROOT SET:  [77] 

BASE SET:  [77, 15, 66, 8] 

Page Rankings:
Page No	Hub score
15	0.3845864380372763	

77	0.3660254037844386	

8	0.16489956693907262	

66	0.08448859123921251	

Page Rankings:
Page No	Auth score
66	0.3660254037844387	

8	0.30009784679806367	

77	0.1875382167312628	

15	0.14633853268623478	



Code for generating documentation

In [ ]:
# !pip install pdoc3

In [ ]:
# file1 = "/content/ir_a2b_hits-2.py"

# from pdoc import text
# # import file
# doc = open("HITS_doc.md", "w")
# doc.write(text(file1))
# # finding_all_unique_words_and_freq
# doc.close()

In [ ]:
# file1 = "/content/ir_a2a_pagerank.py"

# from pdoc import text
# # import file
# doc = open("2A-PageRank_doc.md", "w")
# doc.write(text(file1))
# # finding_all_unique_words_and_freq
# doc.close()